In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from datetime import datetime


In [17]:
def process_single_image(image_path):
    """Process a single image using the original logic."""
    # Read the image with unchanged flag
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if image is None:
        return None, None, 0
    
    # Convert to 3-channel if it's single channel
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    
    # Create a new image
    new_image = np.zeros(image.shape, image.dtype)
    alpha = 3.5
    beta = 0
    
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            for c in range(image.shape[2]):
                new_image[y,x,c] = np.clip(alpha*image[y,x,c] + beta, 0, 255)
                
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    
    threshold = 50
     
    mask = gray_image > threshold
    new_image[mask] = [0, 255, 0]  # Set to green
    
    # Convert to RGB for processing
    image_rgb = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
    hsv_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
    
    # Define range for green color in HSV
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
    
    # Find contours
    contours, _ = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw bounding squares on the original RGB image
    annotated_image = image_rgb.copy()
    
    dot_info = []
    for i, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(annotated_image, (x, y), (x + w, y + h), (255, 255, 0), 2)
        
        dot_info.append({
            'dot_id': i + 1,
            'position': {'x': int(x + w/2), 'y': int(y + h/2)},
            'width': w,
            'height': h,
            'area': int(cv2.contourArea(contour))
        })
    
    # Convert back to BGR for saving
    annotated_image_bgr = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
    
    return annotated_image_bgr, dot_info, len(contours)

In [18]:
def process_folder(input_folder, output_folder, json_output_path):
    """Process all images in a folder and save results."""
    os.makedirs(output_folder, exist_ok=True)
    
    results = {
        'processing_date': datetime.now().isoformat(),
        'total_images_processed': 0,
        'images': {}
    }
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.tif', '.tiff')):
            image_path = os.path.join(input_folder, filename)
            print(f"Processing {filename}...")
            
            try:
                annotated_image, dot_info, dot_count = process_single_image(image_path)
                
                if annotated_image is None:
                    print(f"Failed to process {filename}")
                    continue
                
                # Save the annotated image as TIFF with maximum quality
                output_filename = os.path.join(output_folder, f"annotated_{filename}")
                # Save with original bit depth and compression
                cv2.imwrite(output_filename, 
                          annotated_image,
                          [cv2.IMWRITE_TIFF_COMPRESSION, 1])  # Use LZW compression for better quality
                
                results['images'][filename] = {
                    'filename': filename,
                    'annotated_filename': f"annotated_{filename}",
                    'dot_count': dot_count,
                    'dots': dot_info
                }
                
                results['total_images_processed'] += 1
                
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                continue
    
    with open(json_output_path, 'w') as f:
        json.dump(results, f, indent=4)
    
    return results

In [19]:
# Example usage
if __name__ == "__main__":
    input_folder = "/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/merged_pml_images_High_Arsenic"  # Replace with your input folder path
    output_folder = "/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/merged_pml_images_High_Arsenic_annotations"  # Replace with your output folder path
    json_output_path = "/Users/pallavisingh/Library/CloudStorage/OneDrive-SharedLibraries-DalhousieUniversity/Priyadharshini Sridharan - Images from Dellaire Lab/merged_pml_images_High_Arsenic_annotations/merged_pml_images_High_Arsenic_annotations.json "
    
    results = process_folder(input_folder, output_folder, json_output_path)
    print(f"Processed {results['total_images_processed']} images")
    print(f"Results saved to {json_output_path}")

Processing flattened_position_10_C1.tif...
Processing flattened_position_6_C1.tif...
Processing flattened_position_34_C1.tif...
Processing flattened_position_26_C1.tif...
Processing flattened_position_38_C1.tif...
Processing flattened_position_4_C1.tif...
Processing flattened_position_8_C1.tif...
Processing flattened_position_12_C1.tif...
Processing flattened_position_24_C1.tif...
Processing flattened_position_28_C1.tif...
Processing flattened_position_36_C1.tif...
Processing flattened_position_32_C1.tif...
Processing flattened_position_20_C1.tif...
Processing flattened_position_16_C1.tif...
Processing flattened_position_22_C1.tif...
Processing flattened_position_30_C1.tif...
Processing flattened_position_2_C1.tif...
Processing flattened_position_18_C1.tif...
Processing flattened_position_14_C1.tif...
Processing flattened_position_35_C1.tif...
Processing flattened_position_39_C1.tif...
Processing flattened_position_27_C1.tif...
Processing flattened_position_11_C1.tif...
Processing flat